In [132]:
import pandas as pd
import os
import json
import requests
from pprint import pprint
import numpy as np

In [133]:
def load_api_keys(file_path):
    """
    Load API keys from a JSON file.
    """
    with open(file_path, 'r') as file:
        keys = json.load(file)
    return keys

In [134]:
keys = load_api_keys('../data/api_keys.json')

In [135]:
def yelp_search(api_key, lat, lon):
    """
    Perform a Yelp search near the given latitude and longitude.
    """
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {'Authorization': f'Bearer {api_key}'}
    params = {
        'term': 'restaurant',
        'latitude': lat,
        'longitude': lon,
        'radius': 1609,  # Convert back to meters for Yelp's radius parameter
        'limit': 50  # Yelp API allows up to 50 results per request
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error: Unable to fetch data from Yelp API. Status Code: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error
    

    data = response.json()

    # Create DataFrame from Yelp API response
    df = pd.DataFrame([{
        'Name': business['name'],
        'Business ID': business.get('id'),
        'Address': ', '.join(business['location'].get('display_address', [])),
        'Rating': business.get('rating'),
        'Total Ratings': business.get('review_count'),
        'Price': business.get('price', None),  # Price may not be available for all businesses
        'Cuisine': business.get('categories')
    } for business in data.get('businesses', [])])
    
    return df

In [136]:
df_points = pd.read_csv('../data/nola.csv', header=None, names=['Latitude', 'Longitude'],skiprows=1)

# Initialize an empty DataFrame to store Yelp results


In [137]:
df_points

,Latitude,Longitude
0,29.956970,-90.112957
1,30.011266,-90.077282
2,29.914587,-90.018549
3,29.975803,-90.046806
4,29.962644,-90.097513
...,...,...
102,29.909259,-90.003386
103,30.001888,-90.003923
104,30.018291,-90.036028
105,29.951151,-90.009855


In [138]:
combined_df = pd.DataFrame({
    'Name': ['existing restaurant'],
    'Business ID': ['id1'],
    'Address': ['100 Existing St'],
    'Rating': [0],
    'Total Ratings': [0],
    'Price': ['$'],
    'Cuisine': [None]
})

In [139]:
for i, (index, row) in enumerate(df_points.iterrows()):
    if i >= 5:  # Stop after 5 iterations
        break    
    
    latitude, longitude = row['Latitude'], row['Longitude']

    print("hey",latitude,longitude)
    # Search Yelp using the latitude and longitude
    yelp_df = yelp_search(keys['yelp_api_key'], latitude, longitude)
    yelp_df['Name'] = yelp_df['Name'].str.lower()

    print(len(yelp_df))
    combined_df = pd.concat([combined_df, yelp_df[~yelp_df['Name'].isin(combined_df['Name'])]], ignore_index=True)

    i+=1


hey 29.956969553435577 -90.1129573456206
50
hey 30.011266383314485 -90.07728191476632
22
hey 29.91458695905421 -90.01854855648119
50
hey 29.975802650965527 -90.04680638143522
50
hey 29.96264442348628 -90.09751262655034
50


In [140]:
combined_df.head(n=20)

,Name,Business ID,Address,Rating,Total Ratings,Price,Cuisine
0,existing restaurant,id1,100 Existing St,0.0,0,$,None
1,nice guys nola,AZJpX5slpbZUTU9INbwu0g,"7910 Earhart Blvd, New Orleans, LA 70125",3.2,460,$$,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':..."
2,que pasta nola,zznZqH9CiAznbkV6fXyHWA,"500 S Telemachus St, New Orleans, LA 70119",4.9,15,None,"[{'alias': 'popuprestaurants', 'title': 'Pop-U..."
3,luca eats,854d-ziK_hQFmbNdvF3ZsA,"7329 Cohn St, New Orleans, LA 70118",4.5,201,$,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias..."
4,boucherie,rsMbMeW7IbXxPFh4WwbddQ,"8115 Jeannette St, New Orleans, LA 70118",4.3,627,$$,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia..."
5,asuka sushi & hibachi,HV7AHORTkLqIfhR-tpRmzg,"4600 Washington Ave, New Orleans, LA 70125",4.4,343,$$,"[{'alias': 'japanese', 'title': 'Japanese'}, {..."
6,mikimoto,5rScKeYVjFk8VahsvJv9QA,"3301 S Carrollton Ave, New Orleans, LA 70118",3.9,469,$$,"[{'alias': 'japanese', 'title': 'Japanese'}, {..."
7,ye olde college inn,yY3kq6_2m1FyY9MgaiHgoQ,"3000 S Carrollton Ave, New Orleans, LA 70118",4.2,272,$$,"[{'alias': 'southern', 'title': 'Southern'}, {..."
8,gloria's restaurant,QyO7tZ9ECNq162V53PUzmA,"3101 Broadway St, New Orleans, LA 70125",5.0,2,None,"[{'alias': 'breakfast_brunch', 'title': 'Break..."
9,bspot nola,n3OesNK3bdBoLT7XefiOuQ,"3150 Calhoun St, New Orleans, LA 70125",4.1,106,None,"[{'alias': 'soulfood', 'title': 'Soul Food'}, ..."


In [141]:
import pymysql

import mysql.connector

In [153]:
import pymysql

# Database connection details
host = "database-1.c74gs48gc437.us-east-1.rds.amazonaws.com"
port = 3306
dbname = "restaurants"
user = "admin"
password = "Banker2b!"


    # Establish a connection to the database
conn = pymysql.connect(host=host, user=user, port=port, password=password, database=dbname)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Execute the SQL query to get the list of databases
cursor.execute("SHOW DATABASES;")

# Fetch all the results
databases = cursor.fetchall()

# Print out the databases
print("Databases:")
for db in databases:
    print(db[0])



Databases:
information_schema
mysql
performance_schema
restaurants
sys


In [146]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS restaurant_info (
        name VARCHAR(255),
        business_id VARCHAR(255),
        address VARCHAR(255),
        rating FLOAT,
        total_ratings INT,
        price VARCHAR(10),
        cuisine TEXT
    );
    """
cursor.execute(create_table_query)
conn.commit()
print("Table 'restaurant_info' created successfully.")


Table 'restaurant_info' created successfully.


In [147]:
combined_df

,Name,Business ID,Address,Rating,Total Ratings,Price,Cuisine
0,existing restaurant,id1,100 Existing St,0.0,0,$,None
1,nice guys nola,AZJpX5slpbZUTU9INbwu0g,"7910 Earhart Blvd, New Orleans, LA 70125",3.2,460,$$,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':..."
2,que pasta nola,zznZqH9CiAznbkV6fXyHWA,"500 S Telemachus St, New Orleans, LA 70119",4.9,15,None,"[{'alias': 'popuprestaurants', 'title': 'Pop-U..."
3,luca eats,854d-ziK_hQFmbNdvF3ZsA,"7329 Cohn St, New Orleans, LA 70118",4.5,201,$,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias..."
4,boucherie,rsMbMeW7IbXxPFh4WwbddQ,"8115 Jeannette St, New Orleans, LA 70118",4.3,627,$$,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia..."
...,...,...,...,...,...,...,...
191,mid-city yacht club,jrGaVL6KIwYpwivse_eKXw,"440 S St Patrick St, New Orleans, LA 70119",4.3,166,$$,"[{'alias': 'sportsbars', 'title': 'Sports Bars..."
192,foodies destination,X2W9OyDwI5oKK8ZpyUwxRQ,"3324 Transcontinental Dr, New Orleans, LA 70112",4.6,19,$$,"[{'alias': 'foodtrucks', 'title': 'Food Trucks..."
193,anita's restaurant,kXbzA3rrsW32N78rrLovgA,"2122 Tulane Ave, New Orleans, LA 70112",4.0,92,$,"[{'alias': 'tradamerican', 'title': 'American'..."
194,mr everything cafe,FiWGTyWJs4p4dLQc8P6-pg,"400 S Broad, Ste A, New Orleans, LA 70119",3.3,77,$,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias..."


In [113]:
print(combined_df.dtypes)
print(combined_df['Cuisine'].head(50))



Name              object
Business ID       object
Address           object
Rating           float64
Total Ratings      int64
Price             object
Cuisine           object
dtype: object
0                                                  None
1     [{'alias': 'cafes', 'title': 'Cafes'}, {'alias...
2     [{'alias': 'soulfood', 'title': 'Soul Food'}, ...
3     [{'alias': 'bars', 'title': 'Bars'}, {'alias':...
4     [{'alias': 'tacos', 'title': 'Tacos'}, {'alias...
5            [{'alias': 'mexican', 'title': 'Mexican'}]
6     [{'alias': 'breakfast_brunch', 'title': 'Break...
7     [{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...
8     [{'alias': 'japanese', 'title': 'Japanese'}, {...
9     [{'alias': 'southern', 'title': 'Southern'}, {...
10    [{'alias': 'breakfast_brunch', 'title': 'Break...
11    [{'alias': 'chicken_wings', 'title': 'Chicken ...
12    [{'alias': 'breakfast_brunch', 'title': 'Break...
13    [{'alias': 'bars', 'title': 'Bars'}, {'alias':...
14    [{'alias': 'japanese'

In [148]:
combined_df['Cuisine'] = combined_df['Cuisine'].apply(lambda x: json.dumps(x) if isinstance(x, list) else None)


In [149]:
data_tuples = [
    tuple(None if isinstance(x, (np.float64, np.ndarray)) and np.isnan(x) else x for x in row)
    for row in combined_df.iloc[1:].itertuples(index=False, name=None)  # Skip the first row
]

In [150]:
data_tuples

[('nice guys nola',
  'AZJpX5slpbZUTU9INbwu0g',
  '7910 Earhart Blvd, New Orleans, LA 70125',
  3.2,
  460,
  '$$',
  '[{"alias": "bars", "title": "Bars"}, {"alias": "tradamerican", "title": "American"}, {"alias": "chicken_wings", "title": "Chicken Wings"}]'),
 ('que pasta nola',
  'zznZqH9CiAznbkV6fXyHWA',
  '500 S Telemachus St, New Orleans, LA 70119',
  4.9,
  15,
  None,
  '[{"alias": "popuprestaurants", "title": "Pop-Up Restaurants"}, {"alias": "cuban", "title": "Cuban"}, {"alias": "honduran", "title": "Honduran"}]'),
 ('luca eats',
  '854d-ziK_hQFmbNdvF3ZsA',
  '7329 Cohn St, New Orleans, LA 70118',
  4.5,
  201,
  '$',
  '[{"alias": "cafes", "title": "Cafes"}, {"alias": "breakfast_brunch", "title": "Breakfast & Brunch"}, {"alias": "tradamerican", "title": "American"}]'),
 ('boucherie',
  'rsMbMeW7IbXxPFh4WwbddQ',
  '8115 Jeannette St, New Orleans, LA 70118',
  4.3,
  627,
  '$$',
  '[{"alias": "bbq", "title": "Barbeque"}, {"alias": "southern", "title": "Southern"}]'),
 ('asuka s

In [115]:
def yelp_review_highlights(api_key, business_id):

    url = f"https://api.yelp.com/v3/businesses/{business_id}/review_highlights?count=5"
    headers = {'Authorization': f'Bearer {api_key}'}

    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: Unable to fetch reviews from Yelp API. Status Code: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error
    
    # Parse the JSON response
    reviews = response.json().get('reviews', [])

    # Create a DataFrame from the reviews data
    reviews_df = pd.DataFrame(reviews)

    return reviews_df

In [120]:
top_5 =combined_df['Business ID'].iloc[1:].head(5)

In [122]:
for i in top_5:
    yelp_review_highlights(keys['yelp_api_key'],i)


Error: Unable to fetch reviews from Yelp API. Status Code: 403
Error: Unable to fetch reviews from Yelp API. Status Code: 403
Error: Unable to fetch reviews from Yelp API. Status Code: 403
Error: Unable to fetch reviews from Yelp API. Status Code: 403
Error: Unable to fetch reviews from Yelp API. Status Code: 403


In [162]:
from db import DBmanager

In [163]:
connection = DBmanager()

Exception ignored in: <function DBmanager.__del__ at 0x110568680>
Traceback (most recent call last):
  File "/Users/benfrankstein/Desktop/restaurant-vibez/backend/collection/db.py", line 18, in __del__
    
    ^
  File "/Users/benfrankstein/Desktop/restaurant-vibez/backend/collection/db.py", line 23, in close
    def close(self):
        ^^^^^^^^^^^
AttributeError: 'DBmanager' object has no attribute 'cursor'


OperationalError: (2003, "Can't connect to MySQL server on 'vibez-database1-instance-1.c74gs48gc437.us-east-1.rds.amazonaws.com' ([Errno 8] nodename nor servname provided, or not known)")

In [127]:
data = connection.load_all_restaurants()

In [129]:
temp = pd.DataFrame(data)

In [130]:
print(temp)

                         0                       1  \
0      ye olde college inn  yY3kq6_2m1FyY9MgaiHgoQ   
1           nice guys nola  AZJpX5slpbZUTU9INbwu0g   
2                boucherie  rsMbMeW7IbXxPFh4WwbddQ   
3                luca eats  854d-ziK_hQFmbNdvF3ZsA   
4           que pasta nola  zznZqH9CiAznbkV6fXyHWA   
..                     ...                     ...   
186         up & adam eatz  Bo_TnA3HGE12c7vJ91UCBA   
187  juan's flying burrito  dGeXdSMah56gEHwZNaRQKA   
188    mid-city yacht club  jrGaVL6KIwYpwivse_eKXw   
189      brocato’s kitchen  uRMqISU-SpXRsx0fiJVhcQ   
190  neyow's xl steakhouse  kSBvn1K5-ciNZw___WBo2w   

                                                2    3    4     5  \
0    3000 S Carrollton Ave, New Orleans, LA 70118  4.2  272    $$   
1        7910 Earhart Blvd, New Orleans, LA 70125  3.2  459    $$   
2        8115 Jeannette St, New Orleans, LA 70118  4.3  627    $$   
3             7329 Cohn St, New Orleans, LA 70118  4.5  200     $   
4     

SyntaxError: invalid syntax (1785361161.py, line 1)